# Loading Packages

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
from hashlib import md5
from IPython.display import display, Markdown, Latex, Code
from scipy.special import logsumexp
import os.path
import pystan
from hashlib import md5

# Preprocessing data

In [8]:
def process_data(data_path, mean_sea_level_path, debug=False):
    print(f"Processing data from files: {data_path}, {mean_sea_level_path}")
    
    data = pd.read_csv(data_path)
    mean_sea_level_data = pd.read_csv(mean_sea_level_path)
    year_means = []
    
    for year in range(1971, 2019):
        year_means.append(np.mean(data.loc[data['Year'] == year]['Water level (mm)'].values))

    if debug:
        plt.title('Data relative to mean sea level')
        plt.show()
        plt.title('mean sea level data')
        plt.plot(mean_sea_level_data.values[:, 0], mean_sea_level_data.values[:, 1])
        plt.show()
    
    # Normalize data
    normalized_data_path = f"{data_path}_n.csv"
    if os.path.isfile(normalized_data_path):
        n_data = pd.read_csv(normalized_data_path)
        n_data['date'] = pd.to_datetime(n_data['date'])
        n_data.set_index('date', inplace=True)
        print(f"Read normalized data from {normalized_data_path}")
    
    else:
        n_data = data.copy()
        print(f"Normalizing {data_path}")
        for i, row in n_data.iterrows():
            if i % 1000 == 0:
                print('\r ready: %.2f%%' % (i / len(n_data) * 100), end=" ")
            n_data.iat[i, 5] = row['Water level (mm)'] - mean_sea_level_data.loc[mean_sea_level_data['Year'] == row['Year']].values[0][1]
    
        # Add date index
        import datetime
        def row_to_timestamp(row):
            year = row['Year']
            month = row['m']
            day = row['d']
            hours = datetime.datetime.strptime(row['Time'], "%H:%M").hour
            date = pd.to_datetime(datetime.datetime(year, month, day, hours))
            return date
    
        n_data['date'] = n_data.apply(row_to_timestamp, axis=1)
        n_data.set_index('date', inplace=True)
        n_data['year'] = n_data.index.year + (n_data.index.dayofyear - 1) / 365 + (n_data.index.hour) / (365 * 24)
    
        n_data = n_data.rename(columns={'Water level (mm)': 'water_level'})
    
        n_data.to_csv(normalized_data_path)
        print(f"\nWrote output to {normalized_data_path}")
    
    monthly_means = n_data.resample('M').mean().dropna()
    yearly_means = n_data.resample('Y').mean().dropna()
    
    nans = np.argwhere(np.isnan(monthly_means['water_level']))
    
    if debug:
        plt.title('Year means')
        plt.plot(yearly_means['year'], yearly_means['water_level'])
        plt.show()
    
        plt.title('Month means')
        plt.plot(monthly_means['year'], monthly_means['water_level'])
        plt.show()
    
    return yearly_means, monthly_means


In [9]:
kemi_data_path = r"/home/prakhar/projects/Finnish_Water_Level/data/Kemi.csv"
kemi_mean_sea_level_path = r"/home/prakhar/projects/Finnish_Water_Level/data/kemi_mw_n2000.csv"

yearly_means, monthly_means = process_data(kemi_data_path, kemi_mean_sea_level_path)

Processing data from files: /home/prakhar/projects/Finnish_Water_Level/data/Kemi.csv, /home/prakhar/projects/Finnish_Water_Level/data/kemi_mw_n2000.csv


/home/prakhar/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/home/prakhar/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3249: DtypeWarning: Columns (9,10) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


Read normalized data from /home/prakhar/projects/Finnish_Water_Level/data/Kemi.csv_n.csv


/home/prakhar/.local/lib/python3.6/site-packages/numpy/core/fromnumeric.py:61: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return bound(*args, **kwds)


In [26]:
k=1

x = list(range(1970, 2019)) * k
y = yearly_means['water_level']
g = sum(list(map(lambda x: [x]*49, [i for i in range(1, k + 1)])), [])

data = {
    'N': k*len(yearly_means),
    'K': k,
    'x': x,
    'y': y,
    'g': g
}

print(data)

{'N': 2, 'K': 1, 'x': [1970, 1971, 1972, 1973, 1974, 1975, 1976, 1977, 1978, 1979, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989, 1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018], 'y': date
2000-12-31   -250.898907
2004-12-31    483.266849
Freq: 4A-DEC, Name: water_level, dtype: float64, 'g': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [5]:
def compile_model(model_code, model_name=None):
    """This will automatically cache models - great if you're just running a
    script on the command line.
    See http://pystan.readthedocs.io/en/latest/avoiding_recompilation.html"""

    code_hash = md5(model_code.encode('ascii')).hexdigest()
    if model_name is None:
        cache_fn = 'cached-model-{}.pkl'.format(code_hash)
    else:
        cache_fn = 'cached-{}-{}.pkl'.format(model_name, code_hash)
    try:
        sm = pickle.load(open(cache_fn, 'rb'))
    except:
        sm = pystan.StanModel(model_code=model_code)
        with open(cache_fn, 'wb') as f:
            pickle.dump(sm, f)
    else:
        print("Using cached StanModel")
    return sm

In [6]:
separate_stan_model = """
data {
  int<lower=0> N; // number of data points
  int<lower=1> K; // number of groups
  vector[N] y;
  vector[N] x;  // years
  int<lower=1,upper=K> g[N];
}
parameters {
  vector[K] alpha;
  vector[K] beta;
  vector<lower=0>[K] sigma;
}
model {
  beta ~ normal(3.2, 10^2);
  y ~ normal(alpha[g] + beta[g].*(x-1980), sigma[g]);
}
generated quantities {
 vector[N] log_lik
 vector[N] y_rep;
 for (i in 1:N) {
   log_lik[i] = normal_lpdf(y[i] | alpha[g[i]] + beta[g[i]] .* (x[i]-1980), sigma[g[i]]);
   y_rep[i] = normal_rng(alpha[g[i]] + beta[g[i]] .* (x[i]-1980), sigma[g[i]]);
 }
}
"""
sm_separate = compile_model(separate_stan_model, model_name="separate")

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_ee6743ee772b9470575a4e99cd8eeeba NOW.


In [12]:
fit = sm_separate.sampling(data)

RuntimeError: Exception: mismatch in dimension declared and found in context; processing stage=data initialization; variable name=x; position=0; dims declared=(2); dims found=(49)  (in 'unknown file name' at line 6)


In [ ]:
summary = fit_summary(fit)
display(summary)